In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:36424 Dashboard: http://127.0.0.1:41189/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [5]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tau_x_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tau_x').\
sel(yu_ocean=arange(latS,latN,1), method='nearest').\
sel(xu_ocean=arange(lonW,lonE,1), method='nearest')
print(tau_x_z0_1_mod_month)

tau_x_z0_1_mod_month.load()
print(tau_x_z0_1_mod_month)

xu_ocean = tau_x_z0_1_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tau_x_z0_1_mod_month = tau_x_z0_1_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tau_x_z0_1_mod_month)

tau_x_z0_1_mod = tau_x_z0_1_mod_month.groupby('time.year').mean('time')
print(tau_x_z0_1_mod)

tau_x_z0_1_mod = tau_x_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(tau_x_z0_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'tau_x' (time: 3600, yu_ocean: 91, xu_ocean: 91)>
dask.array<getitem, shape=(3600, 91, 91), dtype=float32, chunksize=(1, 91, 91), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * yu_ocean  (yu_ocean) float64 -69.85 -68.94 -67.98 ... 18.22 18.97 19.76
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      i-directed wind stress forcing u-velocity
    units:          N/m^2
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  surface_downward_x_stress
<xarray.DataArray 'tau_x' (time: 3600, yu_ocean: 91, xu_ocean: 91)>
array([[[        nan,         nan,         nan, ..., -0.00218195,
         -0.00484464, -0.00839656],
        [        nan,         nan,         nan, ..., -0.00336236,
         -0.00288484, -0.00330873],
        [      

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'tau_x' (year: 300, yu_ocean: 91, xu_ocean: 91)>
array([[[        nan,         nan,         nan, ..., -0.04309627,
         -0.04022752, -0.03822029],
        [        nan,         nan,         nan, ..., -0.04154721,
         -0.03634525, -0.03200583],
        [        nan,         nan,         nan, ..., -0.02867649,
         -0.02444766, -0.02089914],
        ...,
        [ 0.0084862 ,  0.00640041,  0.00543369, ..., -0.08373239,
         -0.08315251, -0.0822065 ],
        [ 0.00773436,  0.00539236,  0.00441092, ..., -0.0812814 ,
         -0.08080186, -0.08011071],
        [ 0.0066262 ,  0.00407265,  0.00284274, ..., -0.07726189,
         -0.07642002, -0.07584422]],

       [[        nan,         nan,         nan, ..., -0.02755529,
         -0.02327657, -0.01973198],
        [        nan,         nan,         nan, ..., -0.01962488,
         -0.01420565, -0.00925431],
        [        nan,         nan,         nan, ..., -0.0029069 ,
          0.00206205,  0.00650934],


In [6]:
year = tau_x_z0_1_mod.year
lat_1_mod = tau_x_z0_1_mod.yu_ocean
lon_1_mod = tau_x_z0_1_mod.xu_ocean

tau_x_z0_1_trans_mod = tau_x_z0_1_mod
print(np.shape(tau_x_z0_1_trans_mod))
tau_x_z0_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tau_x_z0_1')
tau_x_z0_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tau_x_z0_1')
tau_x_z0_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tau_x_z0_1')
for iid, i in enumerate(lat_1_mod):
    for jid, j in enumerate(lon_1_mod):
        tau_x_z0_1_slope_mod[iid, jid], _, _, tau_x_z0_1_p_value_mod[iid, jid], tau_x_z0_1_std_err_mod[iid, jid] = \
        stats.linregress(year, tau_x_z0_1_trans_mod[:, iid, jid])
    print('lat_1_mod ' + str(np.array(i)))
print(tau_x_z0_1_slope_mod)

(14, 91, 91)
lat_1_mod -69.84906491784008
lat_1_mod -68.94283782351798
lat_1_mod -67.98072752895108
lat_1_mod -66.96083978225538
lat_1_mod -65.88166483095814
lat_1_mod -64.74208746915161
lat_1_mod -64.14940914737063
lat_1_mod -62.918017835627595
lat_1_mod -62.27927927724076
lat_1_mod -60.95584415500419
lat_1_mod -60.27127522056853
lat_1_mod -58.85694367835927
lat_1_mod -58.12745998812029
lat_1_mod -56.62473949913835
lat_1_mod -55.85192873260888
lat_1_mod -55.06514135034383
lat_1_mod -54.26464933342449
lat_1_mod -52.623752879356864
lat_1_mod -51.7840044546795
lat_1_mod -50.931862328421644
lat_1_mod -50.067707258186175
lat_1_mod -49.19193999593566
lat_1_mod -48.304980620980295
lat_1_mod -47.40726784079731
lat_1_mod -45.58142562459152
lat_1_mod -44.654260025008554
lat_1_mod -43.71826708886703
lat_1_mod -42.7739671351677
lat_1_mod -41.821894309122236
lat_1_mod -40.86259569312122
lat_1_mod -39.89663039616952
lat_1_mod -38.92456862330451
lat_1_mod -37.94699072654606
lat_1_mod -36.96448623895

In [7]:
%%time
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tau_x_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tau_x').\
sel(yu_ocean=arange(latS,latN,1), method='nearest').\
sel(xu_ocean=arange(lonW,lonE,1), method='nearest')
print(tau_x_z0_025_mod_month)

tau_x_z0_025_mod_month.load()
print(tau_x_z0_025_mod_month)

xu_ocean = tau_x_z0_025_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tau_x_z0_025_mod_month = tau_x_z0_025_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tau_x_z0_025_mod_month)

tau_x_z0_025_mod = tau_x_z0_025_mod_month.groupby('time.year').mean('time')
print(tau_x_z0_025_mod)

tau_x_z0_025_mod = tau_x_z0_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(tau_x_z0_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'tau_x' (time: 3600, yu_ocean: 91, xu_ocean: 91)>
dask.array<getitem, shape=(3600, 91, 91), dtype=float32, chunksize=(1, 81, 91), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * yu_ocean  (yu_ocean) float64 -70.04 -68.99 -68.04 ... 17.95 18.9 20.08
  * time      (time) object 1958-01-14 12:00:00 ... 2257-12-14 12:00:00
Attributes:
    long_name:      i-directed wind stress forcing u-velocity
    units:          N/m^2
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  surface_downward_x_stress
<xarray.DataArray 'tau_x' (time: 3600, yu_ocean: 91, xu_ocean: 91)>
array([[[        nan,         nan,         nan, ..., -0.00226918,
         -0.00568985, -0.00993468],
        [        nan,         nan,         nan, ..., -0.00364561,
         -0.00299965, -0.00350384],
        [       

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'tau_x' (year: 300, yu_ocean: 91, xu_ocean: 91)>
array([[[            nan,             nan,             nan, ...,
         -4.52644266e-02, -4.05923128e-02, -3.95697020e-02],
        [            nan,             nan,             nan, ...,
         -4.37205918e-02, -3.61278951e-02, -3.20959948e-02],
        [            nan,             nan,             nan, ...,
         -2.76724771e-02, -2.32647285e-02, -1.94133557e-02],
        ...,
        [ 7.93667976e-03,  6.24356931e-03,  5.36894239e-03, ...,
         -8.40301812e-02, -8.32310244e-02, -8.21118429e-02],
        [ 6.59495266e-03,  4.50632768e-03,  4.35364619e-03, ...,
         -8.10888633e-02, -8.10052752e-02, -7.99066573e-02],
        [ 5.96672343e-03,  3.22819850e-03,  1.60228822e-03, ...,
         -7.49519020e-02, -7.35069439e-02, -7.27552995e-02]],

       [[            nan,             nan,             nan, ...,
         -2.71667913e-02, -1.77858528e-02, -1.22443065e-02],
        [            nan,           

In [8]:
lat_025_mod = tau_x_z0_025_mod.yu_ocean
lon_025_mod = tau_x_z0_025_mod.xu_ocean

tau_x_z0_025_trans_mod = tau_x_z0_025_mod
print(np.shape(tau_x_z0_025_trans_mod))
tau_x_z0_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tau_x_z0_025')
tau_x_z0_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tau_x_z0_025')
tau_x_z0_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tau_x_z0_025')
for iid, i in enumerate(lat_025_mod):
    for jid, j in enumerate(lon_025_mod):
        tau_x_z0_025_slope_mod[iid, jid], _, _, tau_x_z0_025_p_value_mod[iid, jid], tau_x_z0_025_std_err_mod[iid, jid] = \
        stats.linregress(year, tau_x_z0_025_trans_mod[:, iid, jid])
    print('lat_025_mod ' + str(np.array(i)))
print(tau_x_z0_025_slope_mod)

(14, 91, 91)
lat_025_mod -70.04194826192298
lat_025_mod -68.98596236987382
lat_025_mod -68.03557506702957
lat_025_mod -66.9795891749804
lat_025_mod -66.02920187213616
lat_025_mod -64.9731343584332
lat_025_mod -64.00419801547245
lat_025_mod -63.000453214324764
lat_025_mod -61.960968741022334
lat_025_mod -61.00623630265623
lat_025_mod -60.02190673823756
lat_025_mod -59.00736214040289
lat_025_mod -57.96200298543457
lat_025_mod -57.02158171167255
lat_025_mod -56.05675532004928
lat_025_mod -55.06717120897921
lat_025_mod -54.052496893222745
lat_025_mod -53.012422566369025
lat_025_mod -51.94666372937093
lat_025_mod -51.0125182145253
lat_025_mod -50.059170111641855
lat_025_mod -48.92249454907395
lat_025_mod -47.92713690122975
lat_025_mod -47.08275570480149
lat_025_mod -46.05154608040137
lat_025_mod -45.000716820340045
lat_025_mod -43.93025481786292
lat_025_mod -43.02321503336839
lat_025_mod -41.91681597060906
lat_025_mod -40.97990278276779
lat_025_mod -40.02948950490317
lat_025_mod -39.0656468

In [9]:
%%time
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tau_x_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tau_x').\
sel(yu_ocean=arange(latS,latN,1), method='nearest').\
sel(xu_ocean=arange(lonW,lonE,1), method='nearest')
print(tau_x_z0_01_mod_month)

tau_x_z0_01_mod_month.load()
print(tau_x_z0_01_mod_month)

xu_ocean = tau_x_z0_01_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tau_x_z0_01_mod_month = tau_x_z0_01_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tau_x_z0_01_mod_month)

tau_x_z0_01_mod = tau_x_z0_01_mod_month.groupby('time.year').mean('time')
print(tau_x_z0_01_mod)

tau_x_z0_01_mod = tau_x_z0_01_mod.sel(year=slice(2004,2017))
print(tau_x_z0_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'tau_x' (time: 396, yu_ocean: 91, xu_ocean: 91)>
dask.array<getitem, shape=(396, 91, 91), dtype=float32, chunksize=(1, 60, 81), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * yu_ocean  (yu_ocean) float64 -70.02 -69.01 -67.99 ... 18.0 19.04 19.98
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      i-directed wind stress forcing u-velocity
    units:          N/m^2
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  surface_downward_x_stress
<xarray.DataArray 'tau_x' (time: 396, yu_ocean: 91, xu_ocean: 91)>
array([[[            nan,             nan,             nan, ...,
          1.52459564e-02,  1.80294234e-02,  1.86329689e-02],
        [            nan,             nan,             nan, ...,
          2.64270511e-02,  2.592

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [10]:
year = tau_x_z0_01_mod.year
lat_01_mod = tau_x_z0_01_mod.yu_ocean
lon_01_mod = tau_x_z0_01_mod.xu_ocean

tau_x_z0_01_trans_mod = tau_x_z0_01_mod
print(np.shape(tau_x_z0_01_trans_mod))
tau_x_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tau_x_z0_01')
tau_x_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tau_x_z0_01')
tau_x_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tau_x_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        tau_x_z0_01_slope_mod[iid, jid], _, _, tau_x_z0_01_p_value_mod[iid, jid], tau_x_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, tau_x_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(tau_x_z0_01_slope_mod)

(14, 91, 91)
lat_01_mod -70.02080115347766
lat_01_mod -69.00705664834604
lat_01_mod -67.99331214321441
lat_01_mod -66.97956763808278
lat_01_mod -66.00806248733164
lat_01_mod -64.99430220674928
lat_01_mod -64.0042209455608
lat_01_mod -62.97776694305129
lat_01_mod -62.007960199967066
lat_01_mod -61.00625549764092
lat_01_mod -60.02192466827543
lat_01_mod -59.007378755945275
lat_01_mod -58.01502712478788
lat_01_mod -56.99436964151279
lat_01_mod -56.000890558457534
lat_01_mod -54.98119819398587
lat_01_mod -53.99376127037148
lat_01_mod -52.98233889038113
lat_01_mod -52.008268394713866
lat_01_mod -51.012524640214004
lat_01_mod -49.994932844070924
lat_01_mod -49.02096313942841
lat_01_mod -48.02755332504395
lat_01_mod -47.01461997706479
lat_01_mod -45.982102204736925
lat_01_mod -45.00071655087839
lat_01_mod -44.00222836794575
lat_01_mod -42.98664811609322
lat_01_mod -42.02833249959079
lat_01_mod -40.97989882381954
lat_01_mod -39.991188364313565
lat_01_mod -38.98795628098951
lat_01_mod -37.97030

In [11]:
tau_x_z0_1_mod_xr = xr.DataArray(tau_x_z0_1_mod.transpose('yu_ocean', 'xu_ocean', 'year'), name='tau_x_z0_1_mod',
                        coords=[lat_1_mod, lon_1_mod, year], 
                        dims=['yu_ocean', 'xu_ocean', 'year'])
print(tau_x_z0_1_mod_xr)
tau_x_z0_1_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_1_mod.nc')

tau_x_z0_1_slope_mod_xr = xr.DataArray(tau_x_z0_1_slope_mod, name='tau_x_z0_1_slope_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_1_slope_mod_xr)
tau_x_z0_1_slope_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_1_slope_mod.nc')

tau_x_z0_1_p_value_mod_xr = xr.DataArray(tau_x_z0_1_p_value_mod, name='tau_x_z0_1_p_value_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_1_p_value_mod_xr)
tau_x_z0_1_p_value_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_1_p_value_mod.nc')

tau_x_z0_1_std_err_mod_xr = xr.DataArray(tau_x_z0_1_std_err_mod, name='tau_x_z0_1_std_err_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_1_std_err_mod_xr)
tau_x_z0_1_std_err_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_1_std_err_mod.nc')



tau_x_z0_025_mod_xr = xr.DataArray(tau_x_z0_025_mod.transpose('yu_ocean', 'xu_ocean', 'year'), name='tau_x_z0_025_mod',
                        coords=[lat_025_mod, lon_025_mod, year], 
                        dims=['yu_ocean', 'xu_ocean', 'year'])
print(tau_x_z0_025_mod_xr)
tau_x_z0_025_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_025_mod.nc')

tau_x_z0_025_slope_mod_xr = xr.DataArray(tau_x_z0_025_slope_mod, name='tau_x_z0_025_slope_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_025_slope_mod_xr)
tau_x_z0_025_slope_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_025_slope_mod.nc')

tau_x_z0_025_p_value_mod_xr = xr.DataArray(tau_x_z0_025_p_value_mod, name='tau_x_z0_025_p_value_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_025_p_value_mod_xr)
tau_x_z0_025_p_value_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_025_p_value_mod.nc')

tau_x_z0_025_std_err_mod_xr = xr.DataArray(tau_x_z0_025_std_err_mod, name='tau_x_z0_025_std_err_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_025_std_err_mod_xr)
tau_x_z0_025_std_err_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_025_std_err_mod.nc')



tau_x_z0_01_mod_xr = xr.DataArray(tau_x_z0_01_mod.transpose('yu_ocean', 'xu_ocean', 'year'), name='tau_x_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yu_ocean', 'xu_ocean', 'year'])
print(tau_x_z0_01_mod_xr)
tau_x_z0_01_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_01_mod.nc')

tau_x_z0_01_slope_mod_xr = xr.DataArray(tau_x_z0_01_slope_mod, name='tau_x_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_01_slope_mod_xr)
tau_x_z0_01_slope_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_01_slope_mod.nc')

tau_x_z0_01_p_value_mod_xr = xr.DataArray(tau_x_z0_01_p_value_mod, name='tau_x_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_01_p_value_mod_xr)
tau_x_z0_01_p_value_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_01_p_value_mod.nc')

tau_x_z0_01_std_err_mod_xr = xr.DataArray(tau_x_z0_01_std_err_mod, name='tau_x_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_x_z0_01_std_err_mod_xr)
tau_x_z0_01_std_err_mod_xr.to_netcdf(output_path + 'v2/tau_x_z0_01_std_err_mod.nc')

<xarray.DataArray 'tau_x_z0_1_mod' (yu_ocean: 91, xu_ocean: 91, year: 14)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [-0.02373722, -0.00054864, -0.02479606, ..., -0.02227117,
         -0.02754112, -0.01391516],
        [-0.0188685 ,  0.00287976, -0.02100866, ..., -0.01688868,
         -0.02221595, -0.00948011],
        [-0.01507827,  0.00542217, -0.01779161, ..., -0.01231843,
         -0.01749383, -0.00595277]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,       

<xarray.DataArray 'tau_x_z0_025_p_value_mod' (yu_ocean: 91, xu_ocean: 91)>
array([[       nan,        nan,        nan, ..., 0.62615023, 0.82603142,
        0.84229175],
       [       nan,        nan,        nan, ..., 0.9639913 , 0.98073972,
        0.75529045],
       [       nan,        nan,        nan, ..., 0.70039892, 0.60520983,
        0.52399646],
       ...,
       [0.0239187 , 0.02711024, 0.03110945, ..., 0.21646298, 0.20469636,
        0.16464096],
       [0.08553678, 0.08422563, 0.07276958, ..., 0.16013437, 0.14566986,
        0.1333832 ],
       [0.35506458, 0.39048535, 0.21376552, ..., 0.142492  , 0.13066794,
        0.13838147]])
Coordinates:
  * yu_ocean  (yu_ocean) float64 -70.04 -68.99 -68.04 ... 17.95 18.9 20.08
  * xu_ocean  (xu_ocean) float64 90.0 91.0 92.0 93.0 ... 177.0 178.0 179.0 180.0
<xarray.DataArray 'tau_x_z0_025_std_err_mod' (yu_ocean: 91, xu_ocean: 91)>
array([[       nan,        nan,        nan, ..., 0.00077108, 0.0007578 ,
        0.00078492],
       [  

In [12]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tau_y_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tau_y').\
sel(yu_ocean=arange(latS,latN,1), method='nearest').\
sel(xu_ocean=arange(lonW,lonE,1), method='nearest')
print(tau_y_z0_1_mod_month)

tau_y_z0_1_mod_month.load()
print(tau_y_z0_1_mod_month)

xu_ocean = tau_y_z0_1_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tau_y_z0_1_mod_month = tau_y_z0_1_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tau_y_z0_1_mod_month)

tau_y_z0_1_mod = tau_y_z0_1_mod_month.groupby('time.year').mean('time')
print(tau_y_z0_1_mod)

tau_y_z0_1_mod = tau_y_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(tau_y_z0_1_mod)

year = tau_y_z0_1_mod.year
lat_1_mod = tau_y_z0_1_mod.yu_ocean
lon_1_mod = tau_y_z0_1_mod.xu_ocean

tau_y_z0_1_trans_mod = tau_y_z0_1_mod
print(np.shape(tau_y_z0_1_trans_mod))
tau_y_z0_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tau_y_z0_1')
tau_y_z0_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tau_y_z0_1')
tau_y_z0_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='tau_y_z0_1')
for iid, i in enumerate(lat_1_mod):
    for jid, j in enumerate(lon_1_mod):
        tau_y_z0_1_slope_mod[iid, jid], _, _, tau_y_z0_1_p_value_mod[iid, jid], tau_y_z0_1_std_err_mod[iid, jid] = \
        stats.linregress(year, tau_y_z0_1_trans_mod[:, iid, jid])
    print('lat_1_mod ' + str(np.array(i)))
print(tau_y_z0_1_slope_mod)

ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tau_y_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tau_y').\
sel(yu_ocean=arange(latS,latN,1), method='nearest').\
sel(xu_ocean=arange(lonW,lonE,1), method='nearest')
print(tau_y_z0_025_mod_month)

tau_y_z0_025_mod_month.load()
print(tau_y_z0_025_mod_month)

xu_ocean = tau_y_z0_025_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tau_y_z0_025_mod_month = tau_y_z0_025_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tau_y_z0_025_mod_month)

tau_y_z0_025_mod = tau_y_z0_025_mod_month.groupby('time.year').mean('time')
print(tau_y_z0_025_mod)

tau_y_z0_025_mod = tau_y_z0_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(tau_y_z0_025_mod)

lat_025_mod = tau_y_z0_025_mod.yu_ocean
lon_025_mod = tau_y_z0_025_mod.xu_ocean

tau_y_z0_025_trans_mod = tau_y_z0_025_mod
print(np.shape(tau_y_z0_025_trans_mod))
tau_y_z0_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tau_y_z0_025')
tau_y_z0_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tau_y_z0_025')
tau_y_z0_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='tau_y_z0_025')
for iid, i in enumerate(lat_025_mod):
    for jid, j in enumerate(lon_025_mod):
        tau_y_z0_025_slope_mod[iid, jid], _, _, tau_y_z0_025_p_value_mod[iid, jid], tau_y_z0_025_std_err_mod[iid, jid] = \
        stats.linregress(year, tau_y_z0_025_trans_mod[:, iid, jid])
    print('lat_025_mod ' + str(np.array(i)))
print(tau_y_z0_025_slope_mod)

ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

tau_y_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='tau_y').\
sel(yu_ocean=arange(latS,latN,1), method='nearest').\
sel(xu_ocean=arange(lonW,lonE,1), method='nearest')
print(tau_y_z0_01_mod_month)

tau_y_z0_01_mod_month.load()
print(tau_y_z0_01_mod_month)

xu_ocean = tau_y_z0_01_mod_month.xu_ocean
xu_ocean_corrected = xu_ocean + 360
tau_y_z0_01_mod_month = tau_y_z0_01_mod_month.assign_coords(xu_ocean=xu_ocean_corrected)
print(tau_y_z0_01_mod_month)

tau_y_z0_01_mod = tau_y_z0_01_mod_month.groupby('time.year').mean('time')
print(tau_y_z0_01_mod)

tau_y_z0_01_mod = tau_y_z0_01_mod.sel(year=slice(2004,2017))
print(tau_y_z0_01_mod)

year = tau_y_z0_01_mod.year
lat_01_mod = tau_y_z0_01_mod.yu_ocean
lon_01_mod = tau_y_z0_01_mod.xu_ocean

tau_y_z0_01_trans_mod = tau_y_z0_01_mod
print(np.shape(tau_y_z0_01_trans_mod))
tau_y_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tau_y_z0_01')
tau_y_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tau_y_z0_01')
tau_y_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='tau_y_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        tau_y_z0_01_slope_mod[iid, jid], _, _, tau_y_z0_01_p_value_mod[iid, jid], tau_y_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, tau_y_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(tau_y_z0_01_slope_mod)

tau_y_z0_1_mod_xr = xr.DataArray(tau_y_z0_1_mod.transpose('yu_ocean', 'xu_ocean', 'year'), name='tau_y_z0_1_mod',
                        coords=[lat_1_mod, lon_1_mod, year], 
                        dims=['yu_ocean', 'xu_ocean', 'year'])
print(tau_y_z0_1_mod_xr)
tau_y_z0_1_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_1_mod.nc')

tau_y_z0_1_slope_mod_xr = xr.DataArray(tau_y_z0_1_slope_mod, name='tau_y_z0_1_slope_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_1_slope_mod_xr)
tau_y_z0_1_slope_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_1_slope_mod.nc')

tau_y_z0_1_p_value_mod_xr = xr.DataArray(tau_y_z0_1_p_value_mod, name='tau_y_z0_1_p_value_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_1_p_value_mod_xr)
tau_y_z0_1_p_value_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_1_p_value_mod.nc')

tau_y_z0_1_std_err_mod_xr = xr.DataArray(tau_y_z0_1_std_err_mod, name='tau_y_z0_1_std_err_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_1_std_err_mod_xr)
tau_y_z0_1_std_err_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_1_std_err_mod.nc')



tau_y_z0_025_mod_xr = xr.DataArray(tau_y_z0_025_mod.transpose('yu_ocean', 'xu_ocean', 'year'), name='tau_y_z0_025_mod',
                        coords=[lat_025_mod, lon_025_mod, year], 
                        dims=['yu_ocean', 'xu_ocean', 'year'])
print(tau_y_z0_025_mod_xr)
tau_y_z0_025_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_025_mod.nc')

tau_y_z0_025_slope_mod_xr = xr.DataArray(tau_y_z0_025_slope_mod, name='tau_y_z0_025_slope_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_025_slope_mod_xr)
tau_y_z0_025_slope_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_025_slope_mod.nc')

tau_y_z0_025_p_value_mod_xr = xr.DataArray(tau_y_z0_025_p_value_mod, name='tau_y_z0_025_p_value_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_025_p_value_mod_xr)
tau_y_z0_025_p_value_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_025_p_value_mod.nc')

tau_y_z0_025_std_err_mod_xr = xr.DataArray(tau_y_z0_025_std_err_mod, name='tau_y_z0_025_std_err_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_025_std_err_mod_xr)
tau_y_z0_025_std_err_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_025_std_err_mod.nc')



tau_y_z0_01_mod_xr = xr.DataArray(tau_y_z0_01_mod.transpose('yu_ocean', 'xu_ocean', 'year'), name='tau_y_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yu_ocean', 'xu_ocean', 'year'])
print(tau_y_z0_01_mod_xr)
tau_y_z0_01_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_01_mod.nc')

tau_y_z0_01_slope_mod_xr = xr.DataArray(tau_y_z0_01_slope_mod, name='tau_y_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_01_slope_mod_xr)
tau_y_z0_01_slope_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_01_slope_mod.nc')

tau_y_z0_01_p_value_mod_xr = xr.DataArray(tau_y_z0_01_p_value_mod, name='tau_y_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_01_p_value_mod_xr)
tau_y_z0_01_p_value_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_01_p_value_mod.nc')

tau_y_z0_01_std_err_mod_xr = xr.DataArray(tau_y_z0_01_std_err_mod, name='tau_y_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yu_ocean', 'xu_ocean'])
print(tau_y_z0_01_std_err_mod_xr)
tau_y_z0_01_std_err_mod_xr.to_netcdf(output_path + 'v2/tau_y_z0_01_std_err_mod.nc')

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'tau_y' (time: 3600, yu_ocean: 91, xu_ocean: 91)>
dask.array<getitem, shape=(3600, 91, 91), dtype=float32, chunksize=(1, 91, 91), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * yu_ocean  (yu_ocean) float64 -69.85 -68.94 -67.98 ... 18.22 18.97 19.76
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      j-directed wind stress forcing v-velocity
    units:          N/m^2
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  surface_downward_y_stress
<xarray.DataArray 'tau_y' (time: 3600, yu_ocean: 91, xu_ocean: 91)>
array([[[            nan,             nan,             nan, ...,
          5.06191887e-02,  4.59233634e-02,  3.89018990e-02],
        [            nan,             nan,             nan, ...,
          4.30913158e-02,  3

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'tau_y' (year: 300, yu_ocean: 91, xu_ocean: 91)>
array([[[        nan,         nan,         nan, ...,  0.11913464,
          0.10347033,  0.09040898],
        [        nan,         nan,         nan, ...,  0.09069312,
          0.07920095,  0.06954017],
        [        nan,         nan,         nan, ...,  0.06277903,
          0.05541954,  0.04877327],
        ...,
        [ 0.02527307,  0.02399113,  0.02123737, ..., -0.032237  ,
         -0.03234946, -0.03251318],
        [ 0.02614895,  0.02464974,  0.02147867, ..., -0.03212149,
         -0.03243042, -0.03255096],
        [ 0.02679433,  0.02520521,  0.0214464 , ..., -0.03099444,
         -0.03163281, -0.03197301]],

       [[        nan,         nan,         nan, ...,  0.1003058 ,
          0.09014867,  0.08140423],
        [        nan,         nan,         nan, ...,  0.07569272,
          0.06787401,  0.06029862],
        [        nan,         nan,         nan, ...,  0.04963041,
          0.04444501,  0.03914403],


...

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'tau_y' (time: 396, yu_ocean: 91, xu_ocean: 91)>
dask.array<getitem, shape=(396, 91, 91), dtype=float32, chunksize=(1, 60, 81), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * yu_ocean  (yu_ocean) float64 -70.02 -69.01 -67.99 ... 18.0 19.04 19.98
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      j-directed wind stress forcing v-velocity
    units:          N/m^2
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  surface_downward_y_stress
<xarray.DataArray 'tau_y' (time: 396, yu_ocean: 91, xu_ocean: 91)>
array([[[        nan,         nan,         nan, ...,  0.07317347,
          0.0695095 ,  0.0689683 ],
        [        nan,         nan,         nan, ...,  0.05917047,
          0.05685536,  0.05669719],
        [        na

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


lat_01_mod -70.02080115347766
lat_01_mod -69.00705664834604
lat_01_mod -67.99331214321441
lat_01_mod -66.97956763808278
lat_01_mod -66.00806248733164
lat_01_mod -64.99430220674928
lat_01_mod -64.0042209455608
lat_01_mod -62.97776694305129
lat_01_mod -62.007960199967066
lat_01_mod -61.00625549764092
lat_01_mod -60.02192466827543
lat_01_mod -59.007378755945275
lat_01_mod -58.01502712478788
lat_01_mod -56.99436964151279
lat_01_mod -56.000890558457534
lat_01_mod -54.98119819398587
lat_01_mod -53.99376127037148
lat_01_mod -52.98233889038113
lat_01_mod -52.008268394713866
lat_01_mod -51.012524640214004
lat_01_mod -49.994932844070924
lat_01_mod -49.02096313942841
lat_01_mod -48.02755332504395
lat_01_mod -47.01461997706479
lat_01_mod -45.982102204736925
lat_01_mod -45.00071655087839
lat_01_mod -44.00222836794575
lat_01_mod -42.98664811609322
lat_01_mod -42.02833249959079
lat_01_mod -40.97989882381954
lat_01_mod -39.991188364313565
lat_01_mod -38.98795628098951
lat_01_mod -37.97030135258621
lat

<xarray.DataArray 'tau_y_z0_025_slope_mod' (yu_ocean: 91, xu_ocean: 91)>
array([[            nan,             nan,             nan, ...,
        -1.46133435e-03, -1.40261429e-03, -1.51868625e-03],
       [            nan,             nan,             nan, ...,
        -1.77324638e-03, -1.80919833e-03, -2.12280729e-03],
       [            nan,             nan,             nan, ...,
        -1.87219385e-03, -2.09430238e-03, -2.22020075e-03],
       ...,
       [-4.77638216e-04, -3.87648368e-04, -2.62895657e-04, ...,
        -3.68584446e-04, -3.79851857e-04, -3.81300687e-04],
       [-4.38750269e-04, -3.44838063e-04, -2.11138681e-04, ...,
        -3.55964573e-04, -3.98639439e-04, -3.79885715e-04],
       [-4.22120416e-04, -3.01485533e-04, -8.18637230e-05, ...,
        -3.47722484e-04, -3.31148880e-04, -3.53244616e-04]])
Coordinates:
  * yu_ocean  (yu_ocean) float64 -70.04 -68.99 -68.04 ... 17.95 18.9 20.08
  * xu_ocean  (xu_ocean) float64 90.0 91.0 92.0 93.0 ... 177.0 178.0 179.0 180.0
<

In [13]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

pme_river_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='pme_river').\
sel(yt_ocean=arange(latS,latN,1), method='nearest').\
sel(xt_ocean=arange(lonW,lonE,1), method='nearest')
print(pme_river_z0_1_mod_month)

pme_river_z0_1_mod_month.load()
print(pme_river_z0_1_mod_month)

xt_ocean = pme_river_z0_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
pme_river_z0_1_mod_month = pme_river_z0_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(pme_river_z0_1_mod_month)

pme_river_z0_1_mod = pme_river_z0_1_mod_month.groupby('time.year').mean('time')
print(pme_river_z0_1_mod)

pme_river_z0_1_mod = pme_river_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(pme_river_z0_1_mod)

year = pme_river_z0_1_mod.year
lat_1_mod = pme_river_z0_1_mod.yt_ocean
lon_1_mod = pme_river_z0_1_mod.xt_ocean

pme_river_z0_1_trans_mod = pme_river_z0_1_mod
print(np.shape(pme_river_z0_1_trans_mod))
pme_river_z0_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='pme_river_z0_1')
pme_river_z0_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='pme_river_z0_1')
pme_river_z0_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='pme_river_z0_1')
for iid, i in enumerate(lat_1_mod):
    for jid, j in enumerate(lon_1_mod):
        pme_river_z0_1_slope_mod[iid, jid], _, _, pme_river_z0_1_p_value_mod[iid, jid], pme_river_z0_1_std_err_mod[iid, jid] = \
        stats.linregress(year, pme_river_z0_1_trans_mod[:, iid, jid])
    print('lat_1_mod ' + str(np.array(i)))
print(pme_river_z0_1_slope_mod)

ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

pme_river_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='pme_river').\
sel(yt_ocean=arange(latS,latN,1), method='nearest').\
sel(xt_ocean=arange(lonW,lonE,1), method='nearest')
print(pme_river_z0_025_mod_month)

pme_river_z0_025_mod_month.load()
print(pme_river_z0_025_mod_month)

xt_ocean = pme_river_z0_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
pme_river_z0_025_mod_month = pme_river_z0_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(pme_river_z0_025_mod_month)

pme_river_z0_025_mod = pme_river_z0_025_mod_month.groupby('time.year').mean('time')
print(pme_river_z0_025_mod)

pme_river_z0_025_mod = pme_river_z0_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(pme_river_z0_025_mod)

lat_025_mod = pme_river_z0_025_mod.yt_ocean
lon_025_mod = pme_river_z0_025_mod.xt_ocean

pme_river_z0_025_trans_mod = pme_river_z0_025_mod
print(np.shape(pme_river_z0_025_trans_mod))
pme_river_z0_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='pme_river_z0_025')
pme_river_z0_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='pme_river_z0_025')
pme_river_z0_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='pme_river_z0_025')
for iid, i in enumerate(lat_025_mod):
    for jid, j in enumerate(lon_025_mod):
        pme_river_z0_025_slope_mod[iid, jid], _, _, pme_river_z0_025_p_value_mod[iid, jid], pme_river_z0_025_std_err_mod[iid, jid] = \
        stats.linregress(year, pme_river_z0_025_trans_mod[:, iid, jid])
    print('lat_025_mod ' + str(np.array(i)))
print(pme_river_z0_025_slope_mod)

ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

pme_river_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='pme_river').\
sel(yt_ocean=arange(latS,latN,1), method='nearest').\
sel(xt_ocean=arange(lonW,lonE,1), method='nearest')
print(pme_river_z0_01_mod_month)

pme_river_z0_01_mod_month.load()
print(pme_river_z0_01_mod_month)

xt_ocean = pme_river_z0_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
pme_river_z0_01_mod_month = pme_river_z0_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(pme_river_z0_01_mod_month)

pme_river_z0_01_mod = pme_river_z0_01_mod_month.groupby('time.year').mean('time')
print(pme_river_z0_01_mod)

pme_river_z0_01_mod = pme_river_z0_01_mod.sel(year=slice(2004,2017))
print(pme_river_z0_01_mod)

year = pme_river_z0_01_mod.year
lat_01_mod = pme_river_z0_01_mod.yt_ocean
lon_01_mod = pme_river_z0_01_mod.xt_ocean

pme_river_z0_01_trans_mod = pme_river_z0_01_mod
print(np.shape(pme_river_z0_01_trans_mod))
pme_river_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='pme_river_z0_01')
pme_river_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='pme_river_z0_01')
pme_river_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='pme_river_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        pme_river_z0_01_slope_mod[iid, jid], _, _, pme_river_z0_01_p_value_mod[iid, jid], pme_river_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, pme_river_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(pme_river_z0_01_slope_mod)

pme_river_z0_1_mod_xr = xr.DataArray(pme_river_z0_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='pme_river_z0_1_mod',
                        coords=[lat_1_mod, lon_1_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(pme_river_z0_1_mod_xr)
pme_river_z0_1_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_1_mod.nc')

pme_river_z0_1_slope_mod_xr = xr.DataArray(pme_river_z0_1_slope_mod, name='pme_river_z0_1_slope_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_1_slope_mod_xr)
pme_river_z0_1_slope_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_1_slope_mod.nc')

pme_river_z0_1_p_value_mod_xr = xr.DataArray(pme_river_z0_1_p_value_mod, name='pme_river_z0_1_p_value_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_1_p_value_mod_xr)
pme_river_z0_1_p_value_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_1_p_value_mod.nc')

pme_river_z0_1_std_err_mod_xr = xr.DataArray(pme_river_z0_1_std_err_mod, name='pme_river_z0_1_std_err_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_1_std_err_mod_xr)
pme_river_z0_1_std_err_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_1_std_err_mod.nc')



pme_river_z0_025_mod_xr = xr.DataArray(pme_river_z0_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='pme_river_z0_025_mod',
                        coords=[lat_025_mod, lon_025_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(pme_river_z0_025_mod_xr)
pme_river_z0_025_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_025_mod.nc')

pme_river_z0_025_slope_mod_xr = xr.DataArray(pme_river_z0_025_slope_mod, name='pme_river_z0_025_slope_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_025_slope_mod_xr)
pme_river_z0_025_slope_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_025_slope_mod.nc')

pme_river_z0_025_p_value_mod_xr = xr.DataArray(pme_river_z0_025_p_value_mod, name='pme_river_z0_025_p_value_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_025_p_value_mod_xr)
pme_river_z0_025_p_value_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_025_p_value_mod.nc')

pme_river_z0_025_std_err_mod_xr = xr.DataArray(pme_river_z0_025_std_err_mod, name='pme_river_z0_025_std_err_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_025_std_err_mod_xr)
pme_river_z0_025_std_err_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_025_std_err_mod.nc')



pme_river_z0_01_mod_xr = xr.DataArray(pme_river_z0_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='pme_river_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(pme_river_z0_01_mod_xr)
pme_river_z0_01_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_01_mod.nc')

pme_river_z0_01_slope_mod_xr = xr.DataArray(pme_river_z0_01_slope_mod, name='pme_river_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_01_slope_mod_xr)
pme_river_z0_01_slope_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_01_slope_mod.nc')

pme_river_z0_01_p_value_mod_xr = xr.DataArray(pme_river_z0_01_p_value_mod, name='pme_river_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_01_p_value_mod_xr)
pme_river_z0_01_p_value_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_01_p_value_mod.nc')

pme_river_z0_01_std_err_mod_xr = xr.DataArray(pme_river_z0_01_std_err_mod, name='pme_river_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(pme_river_z0_01_std_err_mod_xr)
pme_river_z0_01_std_err_mod_xr.to_netcdf(output_path + 'v2/pme_river_z0_01_std_err_mod.nc')

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'pme_river' (time: 3600, yt_ocean: 91, xt_ocean: 91)>
dask.array<getitem, shape=(3600, 91, 91), dtype=float32, chunksize=(1, 91, 91), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -181.5 -180.5 -179.5
  * yt_ocean  (yt_ocean) float64 -70.07 -69.18 -68.23 ... 17.85 19.36 20.16
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      mass flux of precip-evap+river via sbc (liquid, frozen, e...
    units:          (kg/m^3)*(m/sec)
    valid_range:    [-1000000.  1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  water_flux_into_sea_water
<xarray.DataArray 'pme_river' (time: 3600, yt_ocean: 91, xt_ocean: 91)>
array([[[            nan,             nan,             nan, ...,
          1.31010411e-05,  1.27920721e-05,  1.18672060e-05],
        [            nan,             nan,   

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'pme_river' (year: 300, yt_ocean: 91, xt_ocean: 91)>
array([[[            nan,             nan,             nan, ...,
          1.10684314e-05,  6.18813283e-06, -1.00701129e-06],
        [            nan,             nan,             nan, ...,
          1.79659401e-05,  1.05090485e-05,  6.79417462e-06],
        [            nan,             nan,             nan, ...,
          1.43612588e-05,  1.25023880e-05,  8.94369714e-06],
        ...,
        [ 2.94816637e-05,  2.75560105e-05,  2.43981558e-05, ...,
         -4.85968085e-05, -4.85991222e-05, -4.87519246e-05],
        [ 2.81702432e-05,  2.25848707e-05,  1.40597222e-05, ...,
         -4.84307820e-05, -4.89228296e-05, -4.90347302e-05],
        [ 2.75279544e-05,  1.90630726e-05,  5.93738405e-05, ...,
         -4.65004996e-05, -4.77933900e-05, -4.83328913e-05]],

       [[            nan,             nan,             nan, ...,
          1.25437271e-06,  2.24426071e-06,  2.66272764e-06],
        [            nan,       

...

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'pme_river' (time: 396, yt_ocean: 91, xt_ocean: 91)>
dask.array<getitem, shape=(396, 91, 91), dtype=float32, chunksize=(1, 60, 81), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * yt_ocean  (yt_ocean) float64 -70.0 -68.99 -68.01 -67.0 ... 18.04 18.99 20.03
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      mass flux of precip-evap+river via sbc (liquid, frozen, e...
    units:          (kg/m^3)*(m/sec)
    valid_range:    [-1000000.  1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  water_flux_into_sea_water
<xarray.DataArray 'pme_river' (time: 396, yt_ocean: 91, xt_ocean: 91)>
array([[[            nan,             nan,             nan, ...,
          1.63289616e-04,  1.65610778e-04,  1.60800395e-04],
        [            nan,             nan, 

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


lat_01_mod -69.99968147628742
lat_01_mod -68.9859369711558
lat_01_mod -68.01443182040465
lat_01_mod -67.00068731527303
lat_01_mod -65.9869428101414
lat_01_mod -65.01543765939026
lat_01_mod -63.98230575427869
lat_01_mod -63.00048378360056
lat_01_mod -61.98449280805486
lat_01_mod -60.98201976395361
lat_01_mod -59.996941213465746
lat_01_mod -58.981632346702604
lat_01_mod -57.98854233110828
lat_01_mod -57.02160573730968
lat_01_mod -55.972931534839404
lat_01_mod -55.00989047675615
lat_01_mod -54.02315489419197
lat_01_mod -53.01244197002189
lat_01_mod -51.97749098637534
lat_01_mod -50.981067094942794
lat_01_mod -50.02707563213419
lat_01_mod -48.98817403456743
lat_01_mod -47.99411464728449
lat_01_mod -46.9805294268107
lat_01_mod -46.01684637687337
lat_01_mod -44.96536163378834
lat_01_mod -43.96626270846535
lat_01_mod -43.02322376716278
lat_01_mod -41.99119183746466
lat_01_mod -41.01764577943382
lat_01_mod -40.02949555016067
lat_01_mod -39.026820165172495
lat_01_mod -38.00971786316136
lat_01_m

<xarray.DataArray 'pme_river_z0_025_slope_mod' (yt_ocean: 91, xt_ocean: 91)>
array([[            nan,             nan,             nan, ...,
         3.84079466e-07,  2.14894854e-07,  2.47816509e-07],
       [            nan,             nan,             nan, ...,
         7.23251554e-07,  6.32668326e-07,  6.07412365e-07],
       [            nan,             nan,             nan, ...,
         5.39785347e-07,  3.36864629e-07,  2.55257887e-07],
       ...,
       [-4.68006952e-07, -6.11158852e-07, -1.00223074e-06, ...,
        -3.74515526e-08,  1.22326198e-08, -8.75672373e-08],
       [-6.06432851e-07, -8.85008906e-07, -1.47395966e-06, ...,
        -1.14336661e-07, -1.61639604e-07, -2.52009844e-07],
       [-1.42725177e-07, -9.03764729e-07, -1.70952340e-06, ...,
        -2.46745473e-07, -2.47817525e-07, -3.01669486e-07]])
Coordinates:
  * yt_ocean  (yt_ocean) float64 -69.99 -69.04 -67.98 ... 18.07 19.02 19.96
  * xt_ocean  (xt_ocean) float64 90.12 91.12 92.12 93.12 ... 178.1 179.1 180.

In [5]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

evap_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='evap').\
sel(yt_ocean=arange(latS,latN,1), method='nearest').\
sel(xt_ocean=arange(lonW,lonE,1), method='nearest')
print(evap_z0_1_mod_month)

evap_z0_1_mod_month.load()
print(evap_z0_1_mod_month)

xt_ocean = evap_z0_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
evap_z0_1_mod_month = evap_z0_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(evap_z0_1_mod_month)

evap_z0_1_mod = evap_z0_1_mod_month.groupby('time.year').mean('time')
print(evap_z0_1_mod)

evap_z0_1_mod = evap_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(evap_z0_1_mod)

year = evap_z0_1_mod.year
lat_1_mod = evap_z0_1_mod.yt_ocean
lon_1_mod = evap_z0_1_mod.xt_ocean

evap_z0_1_trans_mod = evap_z0_1_mod
print(np.shape(evap_z0_1_trans_mod))
evap_z0_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='evap_z0_1')
evap_z0_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='evap_z0_1')
evap_z0_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='evap_z0_1')
for iid, i in enumerate(lat_1_mod):
    for jid, j in enumerate(lon_1_mod):
        evap_z0_1_slope_mod[iid, jid], _, _, evap_z0_1_p_value_mod[iid, jid], evap_z0_1_std_err_mod[iid, jid] = \
        stats.linregress(year, evap_z0_1_trans_mod[:, iid, jid])
    print('lat_1_mod ' + str(np.array(i)))
print(evap_z0_1_slope_mod)

ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

evap_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='evap').\
sel(yt_ocean=arange(latS,latN,1), method='nearest').\
sel(xt_ocean=arange(lonW,lonE,1), method='nearest')
print(evap_z0_025_mod_month)

evap_z0_025_mod_month.load()
print(evap_z0_025_mod_month)

xt_ocean = evap_z0_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
evap_z0_025_mod_month = evap_z0_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(evap_z0_025_mod_month)

evap_z0_025_mod = evap_z0_025_mod_month.groupby('time.year').mean('time')
print(evap_z0_025_mod)

evap_z0_025_mod = evap_z0_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(evap_z0_025_mod)

lat_025_mod = evap_z0_025_mod.yt_ocean
lon_025_mod = evap_z0_025_mod.xt_ocean

evap_z0_025_trans_mod = evap_z0_025_mod
print(np.shape(evap_z0_025_trans_mod))
evap_z0_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='evap_z0_025')
evap_z0_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='evap_z0_025')
evap_z0_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='evap_z0_025')
for iid, i in enumerate(lat_025_mod):
    for jid, j in enumerate(lon_025_mod):
        evap_z0_025_slope_mod[iid, jid], _, _, evap_z0_025_p_value_mod[iid, jid], evap_z0_025_std_err_mod[iid, jid] = \
        stats.linregress(year, evap_z0_025_trans_mod[:, iid, jid])
    print('lat_025_mod ' + str(np.array(i)))
print(evap_z0_025_slope_mod)

ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

evap_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='evap').\
sel(yt_ocean=arange(latS,latN,1), method='nearest').\
sel(xt_ocean=arange(lonW,lonE,1), method='nearest')
print(evap_z0_01_mod_month)

evap_z0_01_mod_month.load()
print(evap_z0_01_mod_month)

xt_ocean = evap_z0_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
evap_z0_01_mod_month = evap_z0_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(evap_z0_01_mod_month)

evap_z0_01_mod = evap_z0_01_mod_month.groupby('time.year').mean('time')
print(evap_z0_01_mod)

evap_z0_01_mod = evap_z0_01_mod.sel(year=slice(2004,2017))
print(evap_z0_01_mod)

year = evap_z0_01_mod.year
lat_01_mod = evap_z0_01_mod.yt_ocean
lon_01_mod = evap_z0_01_mod.xt_ocean

evap_z0_01_trans_mod = evap_z0_01_mod
print(np.shape(evap_z0_01_trans_mod))
evap_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='evap_z0_01')
evap_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='evap_z0_01')
evap_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='evap_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        evap_z0_01_slope_mod[iid, jid], _, _, evap_z0_01_p_value_mod[iid, jid], evap_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, evap_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(evap_z0_01_slope_mod)

evap_z0_1_mod_xr = xr.DataArray(evap_z0_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='evap_z0_1_mod',
                        coords=[lat_1_mod, lon_1_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(evap_z0_1_mod_xr)
evap_z0_1_mod_xr.to_netcdf(output_path + 'v2/evap_z0_1_mod.nc')

evap_z0_1_slope_mod_xr = xr.DataArray(evap_z0_1_slope_mod, name='evap_z0_1_slope_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_1_slope_mod_xr)
evap_z0_1_slope_mod_xr.to_netcdf(output_path + 'v2/evap_z0_1_slope_mod.nc')

evap_z0_1_p_value_mod_xr = xr.DataArray(evap_z0_1_p_value_mod, name='evap_z0_1_p_value_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_1_p_value_mod_xr)
evap_z0_1_p_value_mod_xr.to_netcdf(output_path + 'v2/evap_z0_1_p_value_mod.nc')

evap_z0_1_std_err_mod_xr = xr.DataArray(evap_z0_1_std_err_mod, name='evap_z0_1_std_err_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_1_std_err_mod_xr)
evap_z0_1_std_err_mod_xr.to_netcdf(output_path + 'v2/evap_z0_1_std_err_mod.nc')



evap_z0_025_mod_xr = xr.DataArray(evap_z0_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='evap_z0_025_mod',
                        coords=[lat_025_mod, lon_025_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(evap_z0_025_mod_xr)
evap_z0_025_mod_xr.to_netcdf(output_path + 'v2/evap_z0_025_mod.nc')

evap_z0_025_slope_mod_xr = xr.DataArray(evap_z0_025_slope_mod, name='evap_z0_025_slope_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_025_slope_mod_xr)
evap_z0_025_slope_mod_xr.to_netcdf(output_path + 'v2/evap_z0_025_slope_mod.nc')

evap_z0_025_p_value_mod_xr = xr.DataArray(evap_z0_025_p_value_mod, name='evap_z0_025_p_value_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_025_p_value_mod_xr)
evap_z0_025_p_value_mod_xr.to_netcdf(output_path + 'v2/evap_z0_025_p_value_mod.nc')

evap_z0_025_std_err_mod_xr = xr.DataArray(evap_z0_025_std_err_mod, name='evap_z0_025_std_err_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_025_std_err_mod_xr)
evap_z0_025_std_err_mod_xr.to_netcdf(output_path + 'v2/evap_z0_025_std_err_mod.nc')



evap_z0_01_mod_xr = xr.DataArray(evap_z0_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='evap_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(evap_z0_01_mod_xr)
evap_z0_01_mod_xr.to_netcdf(output_path + 'v2/evap_z0_01_mod.nc')

evap_z0_01_slope_mod_xr = xr.DataArray(evap_z0_01_slope_mod, name='evap_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_01_slope_mod_xr)
evap_z0_01_slope_mod_xr.to_netcdf(output_path + 'v2/evap_z0_01_slope_mod.nc')

evap_z0_01_p_value_mod_xr = xr.DataArray(evap_z0_01_p_value_mod, name='evap_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_01_p_value_mod_xr)
evap_z0_01_p_value_mod_xr.to_netcdf(output_path + 'v2/evap_z0_01_p_value_mod.nc')

evap_z0_01_std_err_mod_xr = xr.DataArray(evap_z0_01_std_err_mod, name='evap_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(evap_z0_01_std_err_mod_xr)
evap_z0_01_std_err_mod_xr.to_netcdf(output_path + 'v2/evap_z0_01_std_err_mod.nc')

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'evap' (time: 3600, yt_ocean: 91, xt_ocean: 91)>
dask.array<getitem, shape=(3600, 91, 91), dtype=float32, chunksize=(1, 91, 91), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -70.07 -69.18 -68.23 ... 17.85 19.36 20.16
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -181.5 -180.5 -179.5
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      mass flux from evaporation/condensation (>0 enters ocean)
    units:          (kg/m^3)*(m/sec)
    valid_range:    [-1000000.  1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  water_evaporation_flux
<xarray.DataArray 'evap' (time: 3600, yt_ocean: 91, xt_ocean: 91)>
array([[[           nan,            nan,            nan, ...,
         -1.0363424e-05, -1.0381090e-05, -1.0212231e-05],
        [           nan,            nan,            nan, ...,
     

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'evap' (year: 300, yt_ocean: 91, xt_ocean: 91)>
array([[[           nan,            nan,            nan, ...,
         -8.0627788e-06, -7.8320782e-06, -7.5538433e-06],
        [           nan,            nan,            nan, ...,
         -8.5329211e-06, -8.4227895e-06, -8.2905090e-06],
        [           nan,            nan,            nan, ...,
         -8.6074533e-06, -8.3626283e-06, -8.0145919e-06],
        ...,
        [-4.6089353e-05, -4.5551522e-05, -4.6710451e-05, ...,
         -6.1915496e-05, -6.1690931e-05, -6.1619292e-05],
        [-4.4297441e-05, -4.5163619e-05, -4.7758131e-05, ...,
         -6.1871287e-05, -6.1773673e-05, -6.1818057e-05],
        [-3.9098708e-05, -4.1814536e-05, -4.5467415e-05, ...,
         -6.0859009e-05, -6.0777023e-05, -6.0985378e-05]],

       [[           nan,            nan,            nan, ...,
         -6.9166567e-06, -6.7166220e-06, -6.5630829e-06],
        [           nan,            nan,            nan, ...,
         -5.77032

...

<xarray.DataArray 'evap' (time: 396, yt_ocean: 91, xt_ocean: 91)>
dask.array<getitem, shape=(396, 91, 91), dtype=float32, chunksize=(1, 60, 81), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -70.0 -68.99 -68.01 -67.0 ... 18.04 18.99 20.03
  * xt_ocean  (xt_ocean) float64 -270.0 -269.0 -268.0 ... -182.0 -181.0 -180.0
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      mass flux from evaporation/condensation (>0 enters ocean)
    units:          (kg/m^3)*(m/sec)
    valid_range:    [-1000000.  1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  water_evaporation_flux
<xarray.DataArray 'evap' (time: 396, yt_ocean: 91, xt_ocean: 91)>
array([[[           nan,            nan,            nan, ...,
         -6.6462383e-07, -7.5895525e-07, -7.8090659e-07],
        [           nan,            nan,            nan, ...,
   

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


lat_01_mod -69.99968147628742
lat_01_mod -68.9859369711558
lat_01_mod -68.01443182040465
lat_01_mod -67.00068731527303
lat_01_mod -65.9869428101414
lat_01_mod -65.01543765939026
lat_01_mod -63.98230575427869
lat_01_mod -63.00048378360056
lat_01_mod -61.98449280805486
lat_01_mod -60.98201976395361
lat_01_mod -59.996941213465746
lat_01_mod -58.981632346702604
lat_01_mod -57.98854233110828
lat_01_mod -57.02160573730968
lat_01_mod -55.972931534839404
lat_01_mod -55.00989047675615
lat_01_mod -54.02315489419197
lat_01_mod -53.01244197002189
lat_01_mod -51.97749098637534
lat_01_mod -50.981067094942794
lat_01_mod -50.02707563213419
lat_01_mod -48.98817403456743
lat_01_mod -47.99411464728449
lat_01_mod -46.9805294268107
lat_01_mod -46.01684637687337
lat_01_mod -44.96536163378834
lat_01_mod -43.96626270846535
lat_01_mod -43.02322376716278
lat_01_mod -41.99119183746466
lat_01_mod -41.01764577943382
lat_01_mod -40.02949555016067
lat_01_mod -39.026820165172495
lat_01_mod -38.00971786316136
lat_01_m

<xarray.DataArray 'evap_z0_1_std_err_mod' (yt_ocean: 91, xt_ocean: 91)>
array([[           nan,            nan,            nan, ...,
        7.79691724e-08, 7.74421564e-08, 7.46480610e-08],
       [           nan,            nan,            nan, ...,
        7.38070231e-08, 7.49395634e-08, 7.77855346e-08],
       [           nan,            nan,            nan, ...,
        6.79079191e-08, 6.86707928e-08, 7.13274806e-08],
       ...,
       [8.92705629e-08, 1.04401756e-07, 9.76534952e-08, ...,
        1.08998564e-07, 1.07182230e-07, 1.04291696e-07],
       [8.53239056e-08, 8.46041776e-08, 7.83282538e-08, ...,
        1.52188159e-07, 1.49648533e-07, 1.47370889e-07],
       [7.90702851e-08, 9.47202399e-08, 7.60914524e-08, ...,
        1.57432385e-07, 1.58363807e-07, 1.60573428e-07]])
Coordinates:
  * yt_ocean  (yt_ocean) float64 -70.07 -69.18 -68.23 ... 17.85 19.36 20.16
  * xt_ocean  (xt_ocean) float64 90.5 91.5 92.5 93.5 ... 177.5 178.5 179.5 180.5
<xarray.DataArray 'evap_z0_025_mod' (

CPU times: user 3min 15s, sys: 15.8 s, total: 3min 31s
Wall time: 4min 26s
